In [1]:
import yaml
import matplotlib.pyplot as plt
from lvmsurveysim import IFU, Telescope
from lvmsurveysim.schedule import ObservingPlan, Simulator, TileDB
from lvmsurveysim.target import TargetList
from lvmsurveysim.target import Target
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.visualization import MinMaxInterval, PercentileInterval, SqrtStretch, ImageNormalize

In [2]:
with open('./smc_rot.yaml', 'r') as f:
    tgt = yaml.safe_load(f)
tgt

{'SMC': {'coords': [13.1583, -72.8003],
  'region_type': 'ellipse',
  'frame': 'icrs',
  'region_params': {'a': 3.0, 'b': 1.78, 'pa': -45},
  'priority': 30,
  'observatory': 'LCO',
  'telescope': 'LVM-160',
  'max_airmass': 1.75,
  'min_shadowheight': 1.0,
  'exptime': 1200,
  'n_exposures': 9,
  'min_exposures': 3,
  'min_moon_dist': 45,
  'max_lunation': 0.25,
  'overhead': 1.1,
  'tiling_strategy': 'center_first',
  'tile_overlap': 0.0416,
  'group': ['MCs']}}

In [3]:
targets = TargetList(target_file='./smc_rot.yaml')
print('Tiling Survey ...')
tiledb = TileDB(targets)
tiledb.tile_targets()
tiledb.tile_table.write('smc_tiles.fits', format='fits', overwrite=True)

Tiling Survey ...
Tiling target SMC


In [4]:
tbl = tiledb.tile_table
tbl

tile_id,target_index,target,telescope,ra,dec,pa,target_priority,tile_priority,airmass_limit,lunation_limit,hz_limit,moon_distance_limit,total_exptime,visit_exptime
int64,int64,str3,str7,float64,float64,float64,int64,int64,float64,float64,float64,int64,int64,int64
1000001,0,SMC,LVM-160,17.66710273392139,-75.17576156771024,355.6893729258706,30,1,1.75,0.25,1.0,45,10800,3600
1000002,0,SMC,LVM-160,16.202668839427055,-74.98852989268426,357.08973229728815,30,3,1.75,0.25,1.0,45,10800,3600
1000003,0,SMC,LVM-160,19.004604209267065,-74.93450277542833,354.4118305393602,30,1,1.75,0.25,1.0,45,10800,3600
1000004,0,SMC,LVM-160,14.77395743585471,-74.79225506857156,358.4556740048407,30,4,1.75,0.25,1.0,45,10800,3600
1000005,0,SMC,LVM-160,17.546705731510542,-74.75574967687923,355.80564963785173,30,3,1.75,0.25,1.0,45,10800,3600
1000006,0,SMC,LVM-160,20.300698398276253,-74.68572754789129,353.1743961856275,30,1,1.75,0.25,1.0,45,10800,3600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000100,0,SMC,LVM-160,14.460361477991562,-71.00282326362525,358.7555668946878,30,5,1.75,0.25,1.0,45,10800,3600
1000101,0,SMC,LVM-160,8.912512307553508,-70.74728014031822,4.058075244848213,30,2,1.75,0.25,1.0,45,10800,3600


In [5]:
ifu = IFU.from_config()
lvm160 = Telescope.from_config('LVM-160')

In [6]:
lvm160.plate_scale

<Quantity 348.42060811 arcsec / mm>

In [7]:
# Load SHASSA image
dat, hdr = fits.getdata('010.fl.fits.gz', header=True)
wcs = WCS(hdr)
print(dat.shape)

FileNotFoundError: [Errno 2] No such file or directory: '010.fl.fits.gz'

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=wcs)
norm = ImageNormalize(dat, interval=PercentileInterval(98),
                      stretch=SqrtStretch())
ax.imshow(dat, origin='lower', cmap='cubehelix', norm=norm)
ax.coords[0].set_axislabel('Right Ascension (J2000)')
ax.coords[1].set_axislabel('Declination (J2000)')
ax.grid(color='yellow', ls=':', lw=1)
ax.set_xlim(0, 500)
ax.set_ylim(0, 500)
for p in tbl:
    patch = ifu.get_patch(scale=lvm160.plate_scale, centre=[p['RA'], p['DEC']], pa=p['PA'],
                            edgecolor='None', linewidth=0.0, alpha=0.4, facecolor='b',
                            transform=ax.get_transform('fk5'))[0]
    ax.add_patch(patch)

In [ ]:
smc = Target.from_list('SMC',targets=tgt)
smc

In [ ]:
fig,ax=smc.plot()
fig.set_figheight(3.5)
fig.set_figwidth(4)
ax.invert_xaxis()

In [ ]:
# Load MCELS image
dat, hdr = fits.getdata('mosaic_ha_02_sub.37.fits', header=True)
wcs = WCS(hdr)
print(dat.shape)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=wcs)
vmin, vmax = MinMaxInterval().get_limits(dat)
print(vmax)
norm = ImageNormalize(dat, vmin=0, vmax=150,
                      stretch=SqrtStretch())
ax.imshow(dat, origin='lower', cmap='cubehelix', norm=norm)
ax.coords[0].set_axislabel('Right Ascension (J2000)')
ax.coords[1].set_axislabel('Declination (J2000)')
ax.grid(color='yellow', ls=':', lw=1)
for p in tbl:
    patch = ifu.get_patch(scale=lvm160.plate_scale, centre=[p['RA'], p['DEC']], pa=p['PA'],
                            edgecolor='None', linewidth=0.0, alpha=0.4, facecolor='w',
                            transform=ax.get_transform('fk5'))[0]
    ax.add_patch(patch)